In [ ]:
#AI-generated by ChatGPT
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 📊 Text Analysis & Spam Detection Model Training\n",
    "### Final Project — Spam/Fraud Detection\n",
    "\n",
    "This notebook documents:\n",
    "- Loading the spam dataset\n",
    "- Text cleaning\n",
    "- Exploratory data analysis (EDA)\n",
    "- Feature extraction (TF-IDF)\n",
    "- Training an MLP classifier\n",
    "- Evaluating performance\n",
    "- Saving the trained model + vectorizer\n",
    "\n",
    "**This notebook supports the code located in the `/src` and `/models` directories.**"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📥 1. Load Dataset"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [],
   "source": [
    "from datasets import load_dataset\n",
    "import pandas as pd\n",
    "\n",
    "dataset = load_dataset(\"mshenoda/spam-messages\")\n",
    "df = dataset['train'].to_pandas()\n",
    "\n",
    "df.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📊 2. Clean Text"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [],
   "source": [
    "import re\n",
    "import string\n",
    "\n",
    "def clean_text(text):\n",
    "    text = text.lower()\n",
    "    text = re.sub(r\"http\\S+\", \"\", text)\n",
    "    text = text.translate(str.maketrans(\"\", \"\", string.punctuation))\n",
    "    text = re.sub(r\"\\s+\", \" \", text).strip()\n",
    "    return text\n",
    "\n",
    "df['clean_text'] = df['text'].apply(clean_text)\n",
    "df.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔎 3. Simple EDA (Exploratory Data Analysis)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [],
   "source": [
    "df['label'].value_counts()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [],
   "source": [
    "df['char_count'] = df['clean_text'].apply(len)\n",
    "df['char_count'].describe()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {},
   "outputs": [],
   "source": [
    "df['token_count'] = df['clean_text'].apply(lambda x: len(x.split()))\n",
    "df['token_count'].describe()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔧 4. TF-IDF Vectorizer"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {},
   "outputs": [],
   "source": [
    "from sklearn.feature_extraction.text import TfidfVectorizer\n",
    "\n",
    "vectorizer = TfidfVectorizer(max_features=5000)\n",
    "\n",
    "X = vectorizer.fit_transform(df['clean_text']).toarray()\n",
    "y = df['label'].apply(lambda x: 1 if x == 'spam' else 0).values\n",
    "\n",
    "X.shape, y.shape"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## ✂️ 5. Train/Test Split"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "metadata": {},
   "outputs": [],
   "source": [
    "from sklearn.model_selection import train_test_split\n",
    "\n",
    "X_train, X_test, y_train, y_test = train_test_split(\n",
    "    X, y, test_size=0.2, random_state=42, stratify=y\n",
    ")\n",
    "\n",
    "X_train.shape, X_test.shape"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🧠 6. MLP Classifier (PyTorch)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "metadata": {},
   "outputs": [],
   "source": [
    "import torch\n",
    "import torch.nn as nn\n",
    "import torch.optim as optim\n",
    "\n",
    "class MLPClassifier(nn.Module):\n",
    "    def __init__(self, input_dim):\n",
    "        super().__init__()\n",
    "        self.model = nn.Sequential(\n",
    "            nn.Linear(input_dim, 256),\n",
    "            nn.ReLU(),\n",
    "            nn.Dropout(0.3),\n",
    "            nn.Linear(256, 128),\n",  
    "            nn.ReLU(),\n",
    "            nn.Linear(128, 1)\n",
    "        )\n",
    "\n",
    "    def forward(self, x):\n",
    "        return self.model(x)\n",
    "\n",
    "device = 'cuda' if torch.cuda.is_available() else 'cpu'\n",
    "model = MLPClassifier(5000).to(device)\n",
    "\n",
    "criterion = nn.BCEWithLogitsLoss()\n",
    "optimizer = optim.Adam(model.parameters(), lr=1e-3)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔥 7. Training Loop"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "metadata": {},
   "outputs": [],
   "source": [
    "X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)\n",
    "y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1).to(device)\n",
    "\n",
    "for epoch in range(10):\n",
    "    optimizer.zero_grad()\n",
    "    logits = model(X_train_tensor)\n",
    "    loss = criterion(logits, y_train_tensor)\n",
    "    loss.backward()\n",
    "    optimizer.step()\n",
    "    print(f\"Epoch {epoch+1}/10 - Loss: {loss.item():.4f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🧪 8. Evaluation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 10,
   "metadata": {},
   "outputs": [],
   "source": [
    "from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score\n",
    "\n",
    "X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)\n",
    "with torch.no_grad():\n",
    "    preds = torch.sigmoid(model(X_test_tensor)).cpu().numpy().flatten()\n",
    "\n",
    "pred_labels = (preds >= 0.5).astype(int)\n",
    "\n",
    "acc = accuracy_score(y_test, pred_labels)\n",
    "prec = precision_score(y_test, pred_labels)\n",
    "rec = recall_score(y_test, pred_labels)\n",
    "f1 = f1_score(y_test, pred_labels)\n",
    "\n",
    "acc, prec, rec, f1"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 💾 9. Save Model + Vectorizer"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pickle\n",
    "import os\n",
    "\n",
    "MODEL_DIR = \"../models/text/\"\n",
    "os.makedirs(MODEL_DIR, exist_ok=True)\n",
    "\n",
    "torch.save(model.state_dict(), MODEL_DIR + \"text_model_adam.pt\")\n",
    "\n",
    "with open(MODEL_DIR + \"tfidf_vectorizer.pkl\", \"wb\") as f:\n",
    "    pickle.dump(vectorizer, f)\n",
    "\n",
    "print(\"Saved model and vectorizer ✔️\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}
